<a href="https://colab.research.google.com/github/murthy95/colab/blob/master/resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Residual learning for image recognition 
This notebook implements the resnet50 architecture discussed in [this](https://https://arxiv.org/pdf/1512.03385.pdf) paper.

The same is shown in the figure attached below 

<img src="https://cdn-images-1.medium.com/max/1600/1*aq0q7gCvuNUqnMHh4cpnIw.png" width=500>

In [2]:
!nvidia-smi

Mon Jun  3 07:48:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    15W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
#loading the dataset 
import torch 
import torchvision 
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
#make RESNET50 network
import torch.nn as nn
import torch.nn.functional as F

class residual_block(nn.Module):
  def __init__(self, block_id, layers_repeat):
    super(residual_block, self).__init__()
    self.input_maps = 64 * 2**(block_id)
    #use conv to increase the output channels to 256 for concatenation
    self.layers = []
    if not block_id == 0:
      self.layers.append(nn.Conv2d(self.input_maps*2, self.input_maps*4, 1, 
                                 stride=1, padding=0))
    else:
      self.layers.append(nn.Conv2d(self.input_maps, self.input_maps*4, 1, 
                                 stride=1, padding=0))
      
    self.is_repeat = False
    for _ in range(layers_repeat):
      if not self.is_repeat:
        if not block_id == 0:
            self.layers.append(nn.Conv2d(self.input_maps*2, self.input_maps, 1,
                                              stride=1, padding=0))  
        else:
          self.layers.append(nn.Conv2d(self.input_maps, self.input_maps, 1,
                                              stride=1, padding=0))
      else:
#         self.layers.append(nn.conv2d(self.input_maps*4, self.input_maps*4, 1,
#                                           stride=1, padding=0))
        self.layers.append(nn.Conv2d(self.input_maps*4, self.input_maps, 1,
                                              stride=1, padding=0))
        
      self.layers.append(nn.Conv2d(self.input_maps, self.input_maps, 3, 
                                          stride=1, padding=1))
      self.layers.append(nn.Conv2d(self.input_maps, self.input_maps*4, 1,
                                          stride=1, padding=0))
      self.is_repeat = True
     
  def forward(self, x):
    identity = self.layers[0](x)
    counter = 0
    
    for i in range(1,len(self.layers),1):
      counter += 1
      x = F.relu(self.layers[i](x))
      
      if counter % 3 == 0:
        x += identity
        x = F.relu(x)
        identity = x
        print (x.shape)
    
    return x
  
class Resnet50(nn.Module):
  def __init__(self):
    super(Resnet50, self).__init__()
    self.conv1 = nn.Conv2d(3, 64, 7, stride=2)
    self.maxpool1 = nn.MaxPool2d(3, stride=2)
    self.blocks = self.make_layers([3,4,6,3])
    
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.maxpool1(x)
    x = self.blocks(x)
    x = nn.AveragePool(kernel_size=x.shape[2:])(x)
    x = x.view(x.shape[0], -1)
    x = F.relu(nn.Linear(x.shape[-1], 1000)(x))
    x = nn.Linear(1000, 10)(x)
    return x
  
  def make_layers(self, layers_repeat):
    blocks = []
    for i, expansion in enumerate(layers_repeat):
      blocks.append(residual_block(i, expansion))
    return nn.Sequential(*blocks)
    

In [10]:
import torch.optim as optim

#training parameters
n_epochs = 90
alpha = 0.2
batch_size = 64

device = torch.device('cuda:0')
resnet50 = Resnet50().to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)

#training continues 
global_counter = 0
for i in range(n_epochs):
  stats_loss = []
  #make a iter object to feed data to the network
  dataiter = iter(trainloader)
  
  for _ in range(len(trainloader)):
    x, y = next(dataiter)
    x = x.to(device)
    y = y.to(device)
    pred_y = resnet50.forward(x)
    stats_loss.append(loss(pred_y, y))
    
    optimizer.zero_grad()
    loss(pred_y, y).backward()
    optimizer.step()
    
    if global_counter % 200 == 0:
      print('Training loss at iteration {} : {}'.format(global_counter, 
                                                        stats_loss[-1]))
      val_loss = 0
      for j, test_data in enumerate(testloader):
        x_test, y_test = test_data
        val_loss += loss(resnet(x_test), y_test)
     
      print('Test loss at iteration {} : {}'.format(global_counter, 
                                                        val_loss/j))
      

RuntimeError: ignored